# Multi-Agent

## Deps

In [13]:
!pip install --upgrade pip
!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

## OpenAI API Key

In [14]:
api_key = "sk-proj-..."

## Config

### Libs

In [15]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    ToolCallResult, 
    AgentStream,
    ReActAgent
)

### LLM

In [16]:
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=1,
    api_key=api_key
)

### Calculator Agent

In [17]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> float:
    """Divide two numbers"""
    return a / b


calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract, multiply, divide],
    llm=llm,
)

### Poetry Agent

In [18]:
def poet(topic: str) -> str:
    """Generates poems about specific topics"""
    return f"Use this {topic} as prompt for generate a poem"


poetry_agent = ReActAgent(
    name="poetry",
    description="Create poetry that rhymes with numbers",
    system_prompt="Use your tool and generate a poetry",
    tools=[poet],
    llm=llm,
)

### Workflow

In [19]:
agent = AgentWorkflow(agents=[calculator_agent, poetry_agent], root_agent="calculator")

## Run/Thought

In [21]:
handler = agent.run(user_msg="Solve ((2 + 2) * 2 / 100) - 25 and write a poem")

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
        print("")
    elif isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)

resp = await handler

Thought: The current language of the user is: English. I need to use a tool to help me calculate the given expression before composing a poem. 

First, I'll calculate the value of the expression ((2 + 2) * 2 / 100) - 25 step-by-step. 

1. First, I will compute (2 + 2).
2. Then, I will multiply the result by 2.
3. Next, I will divide that result by 100.
4. Finally, I will subtract 25 from the result.

I'll start with the first step. 

Action: add
Action Input: {"a": 2, "b": 2}Called tool:  add {'a': 2, 'b': 2} => 4

Thought: I have the result of (2 + 2), which is 4. Now I will multiply this result by 2. 
Action: multiply
Action Input: {'a': 4, 'b': 2}Called tool:  multiply {'a': 4, 'b': 2} => 8

Thought: The result of multiplying 4 by 2 is 8. Next, I will divide this result by 100.
Action: divide
Action Input: {'a': 8, 'b': 100}Called tool:  divide {'a': 8, 'b': 100} => 0.08

Thought: The result of dividing 8 by 100 is 0.08. Now I will subtract 25 from this result.
Action: subtract
Acti